In [3]:
!pip install treetaggerwrapper

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 43 kB 2.2 MB/s eta 0:00:01
  Created wheel for treetaggerwrapper: filename=treetaggerwrapper-2.3-py3-none-any.whl size=40773 sha256=e4913ce508174c03b15f69d87f76185a222e042908f5f699cb51415387310adc
  Stored in directory: /home/vincenzo/.cache/pip/wheels/a0/93/50/47079639c52033b2e2b865a59654eea6832068149414cb78a5
Successfully built treetaggerwrapper


In [53]:
import json, time
import pandas as pd
import treetaggerwrapper
import string

### Carichiamo la partizione

In [11]:
with open('Partition_pruned_EN.json') as json_file:
    partition = json.load(json_file)

In [12]:
partition_df = pd.DataFrame.from_dict(partition, orient='index')
partition_df = partition_df.reset_index()
partition_df.columns = ['screen_name','community_id']
partition_df

,screen_name,community_id
0,100REScom,6
1,100erneuerbar,6
2,100givers,2
3,2019Biomass,2
4,2020_baseload,1
...,...,...
3137,younousomarjee,4
3138,zabala_eu,2
3139,zeekayinfo3,3
3140,zlegrand,2


### Carichiamo le descrizioni degli utenti

In [13]:
seed = pd.read_csv('seed_incompleto.csv') #Mancano innovatec_spa e italgas
seed = seed.drop(['Unnamed: 0'],axis=1)
seed = seed.rename(columns={"index": "screen_name"})
seed = seed.drop(['followers_count_high_values', 'followers_count_medium_high_values', 'followers_count_medium_low_values', 'followers_count_low_values'],axis=1)
innovatec_spa = {
    'screen_name': 'innovatec_spa',
    'location': 'Milano',
    'friends_count': 282,
    'listed_count': None,
    'followers_count': 2052,
    'created_at': 'Thu May 01 00:00:00 +0000 2014',
    'lang': None,
    'statuses_count': None,
    'description': 'Operiamo nel settore delle energie rinnovabili offrendo soluzioni tecnologicamente all’avanguardia per la produzione, la gestione e la distribuzione di energia.'
}
italgas = {
    'screen_name': 'italgas',
    'location': 'Milano, Lombardia',
    'friends_count': 50,
    'listed_count': 8,
    'followers_count': 2543,
    'created_at': 'Sun Apr 18 13:42:27 +0000 2016',
    'lang': None,
    'statuses_count': None,
    'description': 'Da oltre 180 anni, siamo leader in Italia nella distribuzione del gas naturale. Since 1837, Italian leader in the distribution of natural gas.'
}
seed = seed.append([innovatec_spa,italgas], ignore_index=True)


friends_1st_level = pd.read_csv('data/friends_1st_level_filtered.csv')
friends_1st_level = friends_1st_level.drop(['Unnamed: 0','energy', 'label'],axis=1)

friends_2nd_level = pd.read_csv('friends_2nd_level_EN.csv')
friends_2nd_level = friends_2nd_level.rename(columns={"Unnamed: 0": "screen_name"})
# friends_2nd_level = friends_2nd_level.drop(['followers_count_high_values', 'followers_count_medium_high_values', 'followers_count_medium_low_values', 'followers_count_low_values', 'pred'],axis=1)

In [14]:
seed.shape

(14, 9)

In [15]:
friends_1st_level.shape

(740, 9)

In [16]:
friends_2nd_level.shape

(4620, 11)

In [17]:
all_users = pd.concat([seed, friends_1st_level, friends_2nd_level])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [18]:
all_users.shape

(5374, 11)

In [19]:
all_users = all_users.drop_duplicates(subset='screen_name', keep="first")

### Facciamo la join per associare la partizione al dataframe di tutti gli utenti

In [20]:
all_users_part = all_users.merge(partition_df, left_on='screen_name', right_on='screen_name')

In [21]:
all_users_part

,created_at,description,detected_lang,followers_count,friends_count,lang,listed_count,location,pred,screen_name,statuses_count,community_id
0,Tue Mar 13 08:49:40 +0000 2012,"We produce wind and solar energy, biomass, and...",NaN,1929.0,312.0,NaN,16.0,NaN,NaN,falckrenewables,695,0
1,Tue Mar 23 16:14:19 +0000 2010,Diventiamo l’#energiachecambiatutto. Anche con...,NaN,10235.0,1601.0,NaN,295.0,"Foro Buonaparte 31, Milano",NaN,EdisonNews,12836,0
2,Fri Mar 06 20:20:52 +0000 2009,Profilo ufficiale in italiano di #Enel. Seguic...,NaN,26703.0,5893.0,NaN,293.0,NaN,NaN,EnelGroupIT,19668,0
3,Thu Jun 30 10:52:29 +0000 2011,La responsabilità dell'energia. L'energia dell...,NaN,5238.0,686.0,NaN,94.0,Italia,NaN,TernaSpA,2093,0
4,Thu Oct 25 13:40:33 +0000 2012,Landirenzo: leader nella produzione di impiant...,NaN,621.0,149.0,NaN,23.0,Reggio Emilia,NaN,LandirenzoSpA,1310,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3137,Thu Apr 25 11:44:24 +0000 2013,"Working at the European Commission, DG Environ...",en,21.0,19.0,NaN,2.0,Brussels,1.0,c_fenech,6,4
3138,Thu Mar 19 16:02:36 +0000 2009,"Rapporteur UN CEPA, Coord.Environm. Impl. Revi...",en,466.0,275.0,NaN,21.0,Brussel,1.0,LouisMeuleman,452,4
3139,Sat Jul 28 11:21:02 +0000 2012,WasteReuse European-funded project aims to the...,en,187.0,222.0,NaN,14.0,Brussels,1.0,WasteReuse,543,4
3140,Tue Nov 27 15:32:21 +0000 2012,Making the food chain more efficient to produc...,en,447.0,795.0,NaN,21.0,NaN,1.0,RESFOODProject,812,4


### Uniamo tutti i testi delle descrizioni in base alle community

In [22]:
all_users_part_with_description = all_users_part[all_users_part['description'].notna()]

In [23]:
merged_descriptions_for_communities = all_users_part_with_description[['community_id','description']].groupby(['community_id'])['description'].apply(lambda x: ','.join(x)).reset_index()
merged_descriptions_for_communities

,community_id,description
0,0,"We produce wind and solar energy, biomass, and..."
1,1,Official twitter account of UN Climate Change....
2,2,"Founded by @algore, we’re bringing the world t..."
3,3,News e informazioni dall'account ufficiale del...
4,4,The official account for @EU_Commission Direct...
5,5,"Energy to inspire the world,Total is a major #..."
6,6,World's #1 Renewable Energy News and Informati...
7,7,Largest public-private climate change partners...


In [24]:
merged_descriptions_for_communities_list = merged_descriptions_for_communities['description'].to_list()

### Topic detection over descriptions

In [25]:
import pyLDAvis.sklearn

In [26]:
pyLDAvis.enable_notebook()

In [27]:
#lemmatizzare i testi con tree tagger?

In [28]:
# src: https://github.com/stopwords-iso/stopwords-it/blob/master/stopwords-it.json
stop_words_it = []

lettere_accentate = {
    'à': 'a',
    'è': 'e',
    'ì': 'i',
    'ò': 'o',
    'ù': 'u',
    'é': 'e',
    'ó': 'o',
    'í': 'i'
}

with open("stopwords-it.json") as json_file:  
    stop_words_it = json.load(json_file)

for i in stop_words_it:
    for acc in lettere_accentate:
        if acc in i:
            # aggiunge alle stop word con l'accento le parole equivalenti senza l'accento
            stop_words_it.append(i.replace(acc,lettere_accentate[acc]))
    
stop_words_it = stop_words_it+ ['goo', 'http', 'https', 'com', 'www', 'htm','html', 'php']

In [29]:
# src: https://gist.github.com/sebleier/554280
stop_words_en = []
file = open("stopwords-en.txt")
while 1:
    line = file.readline()
    if not line: break
    stop_words_en.append(line.split()[0])

In [30]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words=stop_words_it+stop_words_en, lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

<>:6: DeprecationWarning: invalid escape sequence \-
<>:6: DeprecationWarning: invalid escape sequence \-
<>:6: DeprecationWarning: invalid escape sequence \-
<ipython-input-30-cde1dcb87f26>:6: DeprecationWarning: invalid escape sequence \-
  token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')


In [31]:
data_vectorized = vectorizer.fit_transform(merged_descriptions_for_communities_list[0].split())

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ahim', 'avr', 'citt', 'dovr', 'far', 'perch', 'perci', 'sar', 'star'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [32]:
NUM_TOPICS = 5

# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print("NumDocuments,NumTopics:",lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS) 

NumDocuments,NumTopics: (1665, 5)


In [33]:
topics = {}
for idx, topic in enumerate(lda_model.components_):
    topics[idx] = [(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-NUM_TOPICS - 1:-1]]

panel_to_save = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')                

/home/vincenzo/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [34]:
panel_to_save

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      150.222153  -72.269554       1        1  27.930411
0       51.163364 -159.968018       2        1  19.038219
3      -73.144554   21.523815       3        1  18.445364
2      -66.410133 -110.606438       4        1  17.326228
4       54.417324   12.443444       5        1  17.259778, topic_info=   Category       Freq      Term      Total  loglift  logprob
7   Default  41.000000  european  41.000000  27.0000  27.0000
5   Default  26.000000    energy  26.000000  26.0000  26.0000
16  Default  14.000000      news  14.000000  25.0000  25.0000
8   Default  10.000000       gas  10.000000  24.0000  24.0000
17  Default  10.000000  official  10.000000  23.0000  23.0000
..      ...        ...       ...        ...      ...      ...
17   Topic5   0.239756  official  10.029558  -1.9769  -5.2047
2    Topic5   0.239811   climate  10.278365  -2.0012  -5.2045
8    Topic5   0.240439       gas  10.573683  -2.0269  -5.2019
16   Topic5   0.239788      news  14.531969  -2.3476  -5.2046
7    Topic5   0.239817  european  41.906949  -3.4065  -5.2045

[162 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         1  0.921478      account
1         1  0.879307       agency
2         2  0.875626      climate
3         3  0.797305      economy
4         1  0.728841      energia
5         5  0.955289       energy
6         2  0.915048  environment
7         1  0.978358     european
8         3  0.945744          gas
9         1  0.856682       global
10        1  0.872360        green
11        1  0.786995     industry
12        2  0.931838       latest
13        5  0.869648       leader
14        4  0.884099         life
15        2  0.896166      markets
16        4  0.963393         news
17        3  0.897348     official
18        2  0.883925       policy
19        3  0.940445     regional
20        4  0.823115      science
21        4  0.890482        solar
22        2  0.905578        space
23        5  0.817353  sustainable
24        3  0.853321         team
25        3  0.862933       tweets
26        4  0.925475        world, R=27, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3, 5])

## Carichiamo i testi dei tweet degli utenti

In [74]:
from nltk.corpus import stopwords
import re

stop_words = stopwords.words('english')

def processTweet(tweet):
    """
    Takes in a string of text, then performs the following:
    1. Removes links, special characters and other bulk cleaning
    2. Returns the tweet text cleaned
    """
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', tweet)
    #Convert @username to AT_USER
    #tweet = re.sub('@[^\s]+','',tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove hashtags
    #tweet = re.sub(r'#\w*', '', tweet)
    # Remove words with 2 or fewer letters
    tweet = re.sub(r'\b\w\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove new line character
    tweet = tweet.replace('\n', ' ')
    # Remove …
    tweet = tweet.replace('…', ' ')
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ')
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF')
    return tweet


In [76]:
import json

timelines_dir = "output4_G_pruned/"

tweets = {}
for us in all_users['screen_name']:
    filename = timelines_dir+"timeline_"+us+".json"
    print(filename, end=" ")
    data = []
    try:
        with open(filename) as json_file:
            for js in json_file:
    #             print(json.loads(js)['tweet']['truncated'])
    #             print(json.loads(js)['tweet']['entities'].keys())
    #                 print(processTweet(json.loads(js)['tweet']['text']))
    #             print(processTweet(json.loads(js)['tweet']['text']))
                data.append(processTweet(json.loads(js)['tweet']['text']))
    #                 data.append(processTweet(json.loads(js)['tweet']['text'])) #.keys())#['text'])
    #             break
        print(len(data))
        tweets[us] = data
    #     break
    # print(data)
    except:
        print("NO DATA FOR",us)

output4_G_pruned/timeline_falckrenewables.json 240
output4_G_pruned/timeline_EdisonNews.json 580
output4_G_pruned/timeline_EnelGroupIT.json 1080
output4_G_pruned/timeline_TernaSpA.json 560
output4_G_pruned/timeline_LandirenzoSpA.json 60
output4_G_pruned/timeline_eni.json 1500
output4_G_pruned/timeline_snam.json 760
output4_G_pruned/timeline_ERGnow.json 1280
output4_G_pruned/timeline_GruppoHera.json 3212
output4_G_pruned/timeline_gruppoiren.json 720
output4_G_pruned/timeline_gruppo_a2a.json 700
output4_G_pruned/timeline_cirgroup.json NO DATA FOR cirgroup
output4_G_pruned/timeline_innovatec_spa.json 20
output4_G_pruned/timeline_italgas.json 440
output4_G_pruned/timeline_intel.json NO DATA FOR intel
output4_G_pruned/timeline_GretaThunberg.json 3204
output4_G_pruned/timeline_MichaelPhelps.json NO DATA FOR MichaelPhelps
output4_G_pruned/timeline_BBCScienceNews.json 2000
output4_G_pruned/timeline_NYTScience.json 3223
output4_G_pruned/timeline_ftchina.json NO DATA FOR ftchina
output4_G_pruned

output4_G_pruned/timeline_ISPRA_Press.json 1580
output4_G_pruned/timeline_GE_Power.json 500
output4_G_pruned/timeline_vtreadgold.json NO DATA FOR vtreadgold
output4_G_pruned/timeline_terredimezzo.json NO DATA FOR terredimezzo
output4_G_pruned/timeline_formichenews.json NO DATA FOR formichenews
output4_G_pruned/timeline_BarklyMayor.json NO DATA FOR BarklyMayor
output4_G_pruned/timeline_NewClimateEcon.json 1680
output4_G_pruned/timeline_DellEMC_CI.json NO DATA FOR DellEMC_CI
output4_G_pruned/timeline_SiemensGamesa.json 780
output4_G_pruned/timeline_Alcoa.json 120
output4_G_pruned/timeline_IEABirol.json 540
output4_G_pruned/timeline_Climate_Rescue.json 100
output4_G_pruned/timeline_Solar_Sister.json 1398
output4_G_pruned/timeline_oppgreen.json 20
output4_G_pruned/timeline_WMBtweets.json 3214
output4_G_pruned/timeline_tuttogreen.json 960
output4_G_pruned/timeline_FerdiGiugliano.json 1560
output4_G_pruned/timeline_AEEnet.json 1200
output4_G_pruned/timeline_jeffmcstras.json NO DATA FOR jeffm

output4_G_pruned/timeline_SPGlobalPlatts.json 2140
output4_G_pruned/timeline_Kyoto_Club.json 620
output4_G_pruned/timeline_EnergyTribune.json NO DATA FOR EnergyTribune
output4_G_pruned/timeline_GreenFilmFest.json NO DATA FOR GreenFilmFest
output4_G_pruned/timeline_EurvenGreeny.json 200
output4_G_pruned/timeline_meetcenter.json NO DATA FOR meetcenter
output4_G_pruned/timeline_TheEIU_Energy.json NO DATA FOR TheEIU_Energy
output4_G_pruned/timeline_Key4biz.json NO DATA FOR Key4biz
output4_G_pruned/timeline_HydrogenCouncil.json 1020
output4_G_pruned/timeline_ocean_climate.json 1600
output4_G_pruned/timeline_Wind___Energy.json 19
output4_G_pruned/timeline_jacopogiliberto.json NO DATA FOR jacopogiliberto
output4_G_pruned/timeline_AP_ClimatePool.json 20
output4_G_pruned/timeline_NewEconomyMag.json NO DATA FOR NewEconomyMag
output4_G_pruned/timeline_GWECGlobalWind.json 1120
output4_G_pruned/timeline_GreenUnivers.json 3201
output4_G_pruned/timeline_antosavarese.json 140
output4_G_pruned/timeline

output4_G_pruned/timeline_ChrisDebien.json 1639
output4_G_pruned/timeline_ZeroUnoWeb.json NO DATA FOR ZeroUnoWeb
output4_G_pruned/timeline_Collaboriamo.json NO DATA FOR Collaboriamo
output4_G_pruned/timeline_GreenEJournal.json NO DATA FOR GreenEJournal
output4_G_pruned/timeline_gioiozzia.json NO DATA FOR gioiozzia
output4_G_pruned/timeline_SolarShirley.json NO DATA FOR SolarShirley
output4_G_pruned/timeline_AlleyOop24.json NO DATA FOR AlleyOop24
output4_G_pruned/timeline_Talesun_Energy.json 877
output4_G_pruned/timeline_TropWATER.json NO DATA FOR TropWATER
output4_G_pruned/timeline_NaftogazUkraine.json 380
output4_G_pruned/timeline_StartUpGET.json NO DATA FOR StartUpGET
output4_G_pruned/timeline_LumosGlobal.json 220
output4_G_pruned/timeline_PierG.json NO DATA FOR PierG
output4_G_pruned/timeline_CES_Tweet.json 80
output4_G_pruned/timeline_impreseambiente.json 20
output4_G_pruned/timeline_noevanhulst.json NO DATA FOR noevanhulst
output4_G_pruned/timeline_morehousegroup.json NO DATA FOR 

output4_G_pruned/timeline_KJChalvatzis.json 1714
output4_G_pruned/timeline_AmbienteRisorse.json NO DATA FOR AmbienteRisorse
output4_G_pruned/timeline_PactEnvironment.json NO DATA FOR PactEnvironment
output4_G_pruned/timeline_EnergyMedFiera.json 20
output4_G_pruned/timeline_PAGExchange.json 239
output4_G_pruned/timeline_EnergieSensibil.json 20
output4_G_pruned/timeline_omniaenergia.json 120
output4_G_pruned/timeline_ClimateTF.json NO DATA FOR ClimateTF
output4_G_pruned/timeline_CareriPaolo.json NO DATA FOR CareriPaolo
output4_G_pruned/timeline_WWF_Svizzera.json NO DATA FOR WWF_Svizzera
output4_G_pruned/timeline_EnecomPotenza.json NO DATA FOR EnecomPotenza
output4_G_pruned/timeline_HubSiracusa.json NO DATA FOR HubSiracusa
output4_G_pruned/timeline_pasqualemerella.json NO DATA FOR pasqualemerella
output4_G_pruned/timeline_GRicci1978.json NO DATA FOR GRicci1978
output4_G_pruned/timeline_susdefItalia.json 320
output4_G_pruned/timeline_canadianenergy.json 380
output4_G_pruned/timeline_Bidet4

...

output4_G_pruned/timeline_ForCitizensEU.json 1980
output4_G_pruned/timeline_EU_Spring.json 360
output4_G_pruned/timeline_SME4SPACE.json 580
output4_G_pruned/timeline_eyesvetlio.json 220
output4_G_pruned/timeline_krejcoj.json 479
output4_G_pruned/timeline_andrasbaneth.json 475
output4_G_pruned/timeline_lampro74.json 119
output4_G_pruned/timeline_Lucille_Ehrhart.json 40
output4_G_pruned/timeline_ceresproject_eu.json 198
output4_G_pruned/timeline_CEPAEuropa.json 100
output4_G_pruned/timeline_agatthaaa.json 300
output4_G_pruned/timeline_MrsSusiMorgan.json 460
output4_G_pruned/timeline_eulawsd.json 200
output4_G_pruned/timeline_edicdonegal.json 60
output4_G_pruned/timeline_karynmo1001.json 959
output4_G_pruned/timeline_EPWashingtonDC.json 580
output4_G_pruned/timeline_GB_RISE.json 40
output4_G_pruned/timeline_eoscpilot.json 179
output4_G_pruned/timeline_MiamiEUCE.json 100
output4_G_pruned/timeline_eucenter.json 180
output4_G_pruned/timeline_marijnvdv78.json 818
output4_G_pruned/timeline_Ier

output4_G_pruned/timeline_jc_stubbs.json 599
output4_G_pruned/timeline_DijkstraHylke.json NO DATA FOR DijkstraHylke
output4_G_pruned/timeline_BaKu_IFSH.json NO DATA FOR BaKu_IFSH
output4_G_pruned/timeline_raphaelbossong.json NO DATA FOR raphaelbossong
output4_G_pruned/timeline_L_deJonge.json NO DATA FOR L_deJonge
output4_G_pruned/timeline_Frontex.json 580
output4_G_pruned/timeline_JudithPoser.json NO DATA FOR JudithPoser
output4_G_pruned/timeline_endless_europe.json NO DATA FOR endless_europe
output4_G_pruned/timeline_SamuelAgini.json NO DATA FOR SamuelAgini
output4_G_pruned/timeline_ThoenissenC.json NO DATA FOR ThoenissenC
output4_G_pruned/timeline_BenDummett.json 820
output4_G_pruned/timeline_josieleblond.json NO DATA FOR josieleblond
output4_G_pruned/timeline_sivieroand.json NO DATA FOR sivieroand
output4_G_pruned/timeline_cbarnard_IDC.json NO DATA FOR cbarnard_IDC
output4_G_pruned/timeline_BoLykkegaard.json NO DATA FOR BoLykkegaard
output4_G_pruned/timeline_SmartGridPro.json NO DAT

output4_G_pruned/timeline_EUinIsrael.json 3206
output4_G_pruned/timeline_VladimirSucha.json 580
output4_G_pruned/timeline_MarsSecretariat.json 180
output4_G_pruned/timeline_JEF_Europe.json 1378
output4_G_pruned/timeline_kkorneliou.json 280
output4_G_pruned/timeline_VFTO_EU.json 280
output4_G_pruned/timeline_EU_ZMK.json 1440
output4_G_pruned/timeline_Effa_ExecDir.json 160
output4_G_pruned/timeline_FoodRevEU.json 300
output4_G_pruned/timeline_AnneNcp.json 140
output4_G_pruned/timeline_ECDC_EU.json 1480
output4_G_pruned/timeline_EP_Legal.json 420
output4_G_pruned/timeline_eurogender.json 999
output4_G_pruned/timeline_EURightsAgency.json 760
output4_G_pruned/timeline_LIFE_CleanUp.json 80
output4_G_pruned/timeline_GREENOMED_EU.json 180
output4_G_pruned/timeline_MedfestProject.json 60
output4_G_pruned/timeline_EurOBIS_VLIZ.json 140
output4_G_pruned/timeline_EULawDataPubs.json 1760
output4_G_pruned/timeline_eurocrowd.json 140
output4_G_pruned/timeline_EESC_INT.json 380
output4_G_pruned/timeli

output4_G_pruned/timeline_falautano.json 276
output4_G_pruned/timeline_accengage.json NO DATA FOR accengage
output4_G_pruned/timeline_eutrade10.json NO DATA FOR eutrade10
output4_G_pruned/timeline_RoryDear.json NO DATA FOR RoryDear
output4_G_pruned/timeline_frontierCities2.json NO DATA FOR frontierCities2
output4_G_pruned/timeline_FABulous_FI.json NO DATA FOR FABulous_FI
output4_G_pruned/timeline_EUPioneers.json NO DATA FOR EUPioneers
output4_G_pruned/timeline_speedupeurope.json NO DATA FOR speedupeurope
output4_G_pruned/timeline_DrPizza.json NO DATA FOR DrPizza
output4_G_pruned/timeline_EuroPressPrize.json NO DATA FOR EuroPressPrize
output4_G_pruned/timeline_ejcnet.json NO DATA FOR ejcnet
output4_G_pruned/timeline_nelsonmau.json NO DATA FOR nelsonmau
output4_G_pruned/timeline_DrupalDaysEU.json NO DATA FOR DrupalDaysEU
output4_G_pruned/timeline_RoxCiupariu.json NO DATA FOR RoxCiupariu
output4_G_pruned/timeline_InnovationEU.json 20
output4_G_pruned/timeline_enterteam.json NO DATA FOR en

output4_G_pruned/timeline_cityofsound.json 1674
output4_G_pruned/timeline_MobilityReports.json NO DATA FOR MobilityReports
output4_G_pruned/timeline_VisionsofEurope.json NO DATA FOR VisionsofEurope
output4_G_pruned/timeline_europeanpress.json NO DATA FOR europeanpress
output4_G_pruned/timeline_EUDirectory.json NO DATA FOR EUDirectory
output4_G_pruned/timeline_PeterSkinnerMEP.json NO DATA FOR PeterSkinnerMEP
output4_G_pruned/timeline_GBBulldogsAFL.json NO DATA FOR GBBulldogsAFL
output4_G_pruned/timeline_markjansen10.json NO DATA FOR markjansen10
output4_G_pruned/timeline_robpiccoli.json NO DATA FOR robpiccoli
output4_G_pruned/timeline_BrendanJordanMN.json NO DATA FOR BrendanJordanMN
output4_G_pruned/timeline_AndriusKubilius.json NO DATA FOR AndriusKubilius
output4_G_pruned/timeline_secure_gas.json NO DATA FOR secure_gas
output4_G_pruned/timeline_dewulfo.json NO DATA FOR dewulfo
output4_G_pruned/timeline_EmissionsEUETS.json NO DATA FOR EmissionsEUETS
output4_G_pruned/timeline_NickyRobert

In [77]:
len(tweets.keys())

3112

In [78]:
cnt = 0
for k,v in tweets.items():
    cnt+=len(v)
print(cnt)

2188103


In [79]:
users_comm = {}
for index,row in all_users_part[['screen_name','community_id']].iterrows():
    users_comm[row['screen_name']] = row['community_id']
print(users_comm)

{'falckrenewables': 0, 'EdisonNews': 0, 'EnelGroupIT': 0, 'TernaSpA': 0, 'LandirenzoSpA': 0, 'eni': 0, 'snam': 5, 'ERGnow': 0, 'GruppoHera': 0, 'gruppoiren': 0, 'gruppo_a2a': 3, 'innovatec_spa': 0, 'italgas': 0, 'GretaThunberg': 0, 'BBCScienceNews': 0, 'NYTScience': 0, 'UNEP': 0, 'Total': 5, 'ENERGY': 0, 'UNFCCC': 1, 'PaoloGentiloni': 5, 'EnvAgency': 0, 'ClimateReality': 2, 'guardianeco': 3, 'HuffPostGreen': 0, 'USATODAYmoney': 0, 'Chevron': 3, 'NBCNewsBusiness': 0, 'PlanetGreen': 3, 'climateprogress': 3, 'OGJOnline': 5, 'IPCC_CH': 5, 'EnvDefenseFund': 3, 'IEA': 0, 'ClimateDesk': 3, 'Connect4Climate': 2, 'ClimateGroup': 5, 'EIAgov': 0, 'REWorld': 6, 'WorldOil': 3, 'insideclimate': 3, 'TheGlobalGoals': 0, 'GreenEnergyNews': 3, 'GreenBiz': 3, 'DeSmogBlog': 3, 'SolarEnergyNews': 0, 'ftenergy': 0, 'SEIA': 3, 'EU_ENV': 4, 'arikring': 2, 'GreenNewsDaily': 3, 'nytclimate': 0, 'IRENA': 0, 'EUEnvironment': 1, 'AWEA': 3, 'PPFEnergia': 3, 'APIenergy': 0, 'resilientplanb': 1, 'EnergyCollectiv': 3,

In [80]:
tweets_partitioned = {}

for k,v in tweets.items():
    if users_comm[k] in tweets_partitioned.keys():
        tmp = tweets_partitioned[users_comm[k]]
        tmp.append(v)
        tweets_partitioned[users_comm[k]] = tmp
    else:
        tweets_partitioned[users_comm[k]] = [v]

In [81]:
tweets_partitioned.keys()

dict_keys([0, 5, 3, 1, 2, 6, 4, 7])

In [82]:
for k,v in tweets_partitioned.items():
    print(k,len(v))

0 98
5 358
3 471
1 173
2 496
6 360
4 1066
7 90


In [83]:
# tweets_partitioned[0][0]

In [84]:
# ' '.join(tweets_partitioned[0][0])

In [85]:
# test_dict = {
#     0: [
#         ['a','b'],['c','d','e'],['f','g']
#     ],
#     1: [
#         ['h','i'],['l','m','n'],['o','p']
#     ] 
# }

In [86]:
treetaggerwrapper.enable_debugging_log()

In [87]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG="en") #, TAGOPT="--debug")

T140080048604992 2020-06-23 09:51:11,890 DEBUG Using treetaggerwrapper.py from /opt/conda/lib/python3.7/site-packages/treetaggerwrapper.py
T140080048604992 2020-06-23 09:51:11,891 DEBUG Found param TAGLANG in paramsdict.
T140080048604992 2020-06-23 09:51:11,892 INFO lang=en
T140080048604992 2020-06-23 09:51:11,892 DEBUG Use default value for param TAGDIR.
T140080048604992 2020-06-23 09:51:11,893 INFO Use previously found TreeTagger directory: /home/vincenzo/tree-tagger-linux
T140080048604992 2020-06-23 09:51:11,894 INFO tagdir=/home/vincenzo/tree-tagger-linux
T140080048604992 2020-06-23 09:51:11,895 INFO tagbin=/home/vincenzo/tree-tagger-linux/bin/tree-tagger
T140080048604992 2020-06-23 09:51:11,895 DEBUG Use default value for param TAGPARFILE.
T140080048604992 2020-06-23 09:51:11,896 INFO tagparfile=/home/vincenzo/tree-tagger-linux/lib/english.par
T140080048604992 2020-06-23 09:51:11,897 DEBUG Use default value for param TAGINENC.
T140080048604992 2020-06-23 09:51:11,897 DEBUG Use def

In [89]:
tweets_partitioned_as_strings = {}

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

# for k,v in test_dict.items():
for k,v in tweets_partitioned.items():
    s = ""
    for i in v: # v e' la lista di stringhe di un certo utente
        s += ' '.join(i)
        s += ' '
    s = s.lower()
    s = s.translate(translator)
    time.sleep(2)
    tags = tagger.tag_text(s)
    tagged_words = treetaggerwrapper.make_tags(tags)
    #print(tagged_words[:1000])
    s_tmp = ""
    for tw in tagged_words:
        if type(tw) != treetaggerwrapper.NotTag and len(tw[2]) > 3:
            #print(tw[2])
            #break
            s_tmp += tw[2] + " "
#     print(s_tmp[:1000])
#     break
    tweets_partitioned_as_strings[k] = s_tmp
    
# tweets_partitioned_as_strings

T140080048604992 2020-06-23 09:52:47,670 DEBUG tag_text with option: numlines=0, tagonly=0, prepronly=0, notagurl=0, tagblanks=0, notagemail=0, notagip=0, notagdns=0, nosgmlsplit=0).
T140080048604992 2020-06-23 09:52:47,671 DEBUG Pre-processing text with internal chunker.
T140080048604992 2020-06-23 09:52:47,671 DEBUG Preparing text for tagger with options tagblanks=0, numlines=0, notagurl=0, notagemail=0, notagip=0, notagdns=0, nosgmlsplit=0).
T140080048604992 2020-06-23 09:52:47,674 DEBUG Identifying SGML tags from within text.
T140080048604992 2020-06-23 09:52:47,682 DEBUG Splitted between SGML tags and others %r.
T140080048604992 2020-06-23 09:52:47,683 DEBUG Replacing blanks by spaces.
T140080048604992 2020-06-23 09:52:48,424 DEBUG Blanks replacement done.
T140080048604992 2020-06-23 09:52:48,425 DEBUG Replacing URLs.
T140080048604992 2020-06-23 09:52:49,732 DEBUG URLs replacement done.
T140080048604992 2020-06-23 09:52:49,733 DEBUG Replacing Emails.
T140080048604992 2020-06-23 09

T140080048604992 2020-06-23 09:58:30,667 DEBUG Blanks replacement done.
T140080048604992 2020-06-23 09:58:30,668 DEBUG Replacing URLs.
T140080048604992 2020-06-23 09:58:34,630 DEBUG URLs replacement done.
T140080048604992 2020-06-23 09:58:34,631 DEBUG Replacing Emails.
T140080048604992 2020-06-23 09:58:37,531 DEBUG Emails replacement done.
T140080048604992 2020-06-23 09:58:37,532 DEBUG Replacing IP addresses.
T140080048604992 2020-06-23 09:58:38,974 DEBUG IP adresses replacement done.
T140080048604992 2020-06-23 09:58:38,975 DEBUG Replacing DNS names.
T140080048604992 2020-06-23 09:58:45,362 DEBUG DNS names replacement done.
T140080048604992 2020-06-23 09:58:45,363 DEBUG Splittint parts of text.
T140080048604992 2020-06-23 09:59:08,800 DEBUG Text preprocessed, parts splitted one by line.
T140080048604992 2020-06-23 09:59:09,793 DEBUG Tagging text.
T140075488397056 2020-06-23 09:59:09,794 INFO Writing starting part to pipe.
T140075488397056 2020-06-23 09:59:09,795 INFO Writing data to p

In [90]:
# tweets_partitioned_as_strings[0]

In [91]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words=stop_words_it+stop_words_en, lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

<>:6: DeprecationWarning: invalid escape sequence \-
<>:6: DeprecationWarning: invalid escape sequence \-
<>:6: DeprecationWarning: invalid escape sequence \-
<ipython-input-91-cde1dcb87f26>:6: DeprecationWarning: invalid escape sequence \-
  token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')


In [92]:
# TOPIC DELLA COMMUNITY 0

data_vectorized = vectorizer.fit_transform(tweets_partitioned_as_strings[0].split())

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ahim', 'avr', 'citt', 'dovr', 'far', 'perch', 'perci', 'sar', 'star'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [93]:
NUM_TOPICS = 3

# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)

# IL NUMERO DI DOCUMENTI QUI INTENDE IL NUMERO DI PAROLE
print("NumDocuments,NumTopics:",lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS) 

NumDocuments,NumTopics: (1220524, 5)


In [94]:
topics = {}
for idx, topic in enumerate(lda_model.components_):
    topics[idx] = [(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-NUM_TOPICS - 1:-1]]

panel_to_save = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

/home/vincenzo/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [95]:
panel_to_save

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      150.222153  -72.269554       1        1  20.314622
3       51.163364 -159.968018       2        1  20.078843
0      -73.144554   21.523815       3        1  19.883057
2      -66.410133 -110.606438       4        1  19.866516
4       54.417324   12.443444       5        1  19.856961, topic_info=      Category         Freq      Term        Total  loglift  logprob
18678  Default  5024.000000     today  5024.000000  30.0000  30.0000
6552   Default  4595.000000    europe  4595.000000  29.0000  29.0000
20385  Default  4226.000000      year  4226.000000  28.0000  28.0000
6058   Default  3804.000000    energy  3804.000000  27.0000  27.0000
20260  Default  3087.000000      work  3087.000000  26.0000  26.0000
...        ...          ...       ...          ...      ...      ...
13947   Topic5   734.463025    planet   735.255053   1.6155  -5.6100
4210    Topic5   734.063154    create   734.855139   1.6155  -5.6105
12103   Topic5   722.080030     month   722.872045   1.6155  -5.6270
4929    Topic5   679.464930   develop   680.256901   1.6155  -5.6878
17238   Topic5   671.992154  solution   672.784155   1.6154  -5.6988

[180 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
183        1  0.999247   action
618        4  0.999322     also
1093       4  0.998848   around
1132       3  0.998840  article
1265       3  0.999541   asylum
...      ...       ...      ...
20267      2  0.999008  working
20273      1  0.999590    world
20313      5  0.999410    would
20385      1  0.999775     year
20424      5  0.998374    young

[150 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3, 5])

In [96]:
# TOPICS OF ALL COMMUNITIES

for k in tweets_partitioned.keys():
    
    data_vectorized = vectorizer.fit_transform(tweets_partitioned_as_strings[k].split())

    NUM_TOPICS = 3

    # Build a Latent Dirichlet Allocation Model
    lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=50, learning_method='online')
    lda_Z = lda_model.fit_transform(data_vectorized)

    # IL NUMERO DI DOCUMENTI QUI INTENDE IL NUMERO DI PAROLE
    print("NumDocuments,NumTopics:",lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS) 

    topics = {}
    for idx, topic in enumerate(lda_model.components_):
        topics[idx] = [(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-NUM_TOPICS - 1:-1]]

    panel_to_save = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

    panel_to_save

NumDocuments,NumTopics: (1220524, 3)


/home/vincenzo/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


NumDocuments,NumTopics: (3968835, 3)


/home/vincenzo/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


NumDocuments,NumTopics: (2564228, 3)


/home/vincenzo/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


NumDocuments,NumTopics: (1519550, 3)


/home/vincenzo/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


KeyboardInterrupt: 